In [1]:
!git clone https://github.com/michaelwwk/appStoreAnalytics.git

Cloning into 'appStoreAnalytics'...
remote: Enumerating objects: 3393, done.
remote: Counting objects: 100% (443/443), done.
remote: Compressing objects: 100% (150/150), done.
remote: Total 3393 (delta 277), reused 435 (delta 273), pack-reused 2950
Receiving objects: 100% (3393/3393), 1.75 MiB | 5.78 MiB/s, done.
Resolving deltas: 100% (2250/2250), done.


In [2]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import numpy as np
import json
from pyspark.sql.functions import split, lower
from pyspark.sql.functions import concat, col, lit
from google.cloud import bigquery, storage
!pip install pyspark
from pyspark.sql import SparkSession
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import nltk
nltk.download('punkt')
import warnings
warnings.filterwarnings("ignore")
import sklearn.preprocessing as pp
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=1bbeb599128f4b056c901d9f302bc03fe07895492e34fd4ed78b9b1925689409
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Start Spark session
spark = SparkSession.builder.master("local").appName("appStoreAnalytics").config('spark.ui.port', '4050').getOrCreate()

In [6]:
# googleAPI_json_path = "/content/drive/MyDrive/Colab Notebooks/big-data-analytics-415801-f8364cc7c7d5.json"
# project_id = "big-data-analytics-415801"
# client = bigquery.Client.from_service_account_json(googleAPI_json_path, project = project_id)

In [7]:
# Define your BigQuery project ID and table ID
project_id = "big-data-analytics-415801"
table_id = 'dev_cleanData.cleanGoogleMain'

# SQL query to fetch the data from the table
sql_query = f"SELECT * FROM {table_id} limit 5000"

# Read data from BigQuery into a DataFrame
googleMain = pd.read_gbq(sql_query, project_id=project_id)

In [8]:
googleMain.head()

,title,description,summary,installs,minInstalls,realInstalls,score,ratings,reviews,price,free,currency,offersIAP,inAppProductPrice,developer,genre,genreId,contentRating,contentRatingDescription,adSupported,released,lastUpdatedOn,version,appId,categories_list,min_inAppProductPrice,max_inAppProductPrice,1starrating_no,2starrating_no,3starrating_no,4starrating_no,5starrating_no
0,CWD Smart,Thanks for choosing CWD Smart app.\r\nYou can ...,CWD Smart can help you to control your Colling...,500+,500,628,2.6,5,0,0.0,True,USD,False,None,Collingwood Lighting,Tools,TOOLS,Everyone,None,False,None,"May 19, 2022",2.24.0,com.collingwood.smarthome,[Tools],None,None,3,0,0,0,2
1,CCSWE App Manager (Root),CCSWE App Manager</b> was designed for users w...,CCSWE App Manager is the ultimate package disa...,"1,000+",1000,1011,3.75,25,6,2.49,False,USD,False,None,Cory Charlton,Tools,TOOLS,Everyone,None,False,"Oct 9, 2020","Sep 24, 2023",6.6.1,com.ccswe.appmanager.root_20201007,[Tools],None,None,6,0,0,6,12
2,Click Counter and Tally Counte,Card Counter is a powerful and easy to use Tal...,Card Counter is a tally counter that makes cou...,"1,000+",1000,4556,3.6,23,3,0.0,True,USD,True,$0.99 per item,AGF,Tools,TOOLS,Everyone,None,True,"Jul 30, 2018","Sep 19, 2021",1.7,com.card.counter,[Tools],0.99,None,4,0,4,4,9
3,Developer Tools,Developer tools</b> is an application for andr...,Developer tools app is a powerful application ...,"10,000+",10000,35012,4.2083335,187,20,0.0,True,USD,False,None,Roxanne Crete,Tools,TOOLS,Everyone,None,True,"Apr 24, 2019","Aug 24, 2023",8.0,rox.developer.tools,[Tools],None,None,19,9,7,25,124
4,EverTranslator ScreenTranslate,"Translate text anytime and everywhere, even yo...",Screen translator - Translate text anytime and...,"100,000+",100000,494071,3.41,2376,103,0.0,True,USD,False,None,Firemaples,Tools,TOOLS,Everyone,None,True,"Dec 29, 2016","Jan 20, 2024",3.1.25,tw.firemaples.onscreenocr,[Tools],None,None,641,131,247,247,1064


In [60]:
googleMain.to_csv("/content/drive/MyDrive/Colab Notebooks/googleMain.csv")

# Read CSV files into PySpark DataFrame
googleMain_sparkDF = spark.read.format("csv") \
                    .option("inferSchema", "true") \
                    .option("header", "true") \
                    .option("multiline", "true") \
                    .option("escape", "\"") \
                    .csv("/content/drive/MyDrive/Colab Notebooks/googleMain.csv")

## Data preparation

In [10]:
filter_list = []

def preprocess_text(text):
    # Tokenize the input text
    tokens = word_tokenize(text)

    # Convert all characters to lower case
    tokens = [t.lower() for t in tokens]

    # Remove Stopwords and filtered words
    stop_words = set(stopwords.words('english'))
    tokens = [t for t in tokens if t not in stop_words and t not in filter_list]

    # Remove Punctuations
    tokens = [t for t in tokens if t not in string.punctuation]

    # Remove Numbers/Numerics
    tokens = [t for t in tokens if not t.isnumeric()]

    # Lemmatize
    lemmatizer = nltk.WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(w) for w in tokens]

    return tokens

In [68]:
text = googleMain['title'] + ' ' + googleMain['description'] + ' ' + googleMain['summary']

In [12]:
tfidf = TfidfVectorizer(tokenizer = preprocess_text, min_df = 2, max_df = 0.7)
tdm = tfidf.fit_transform(text)

In [13]:
tdm

<5000x15414 sparse matrix of type '<class 'numpy.float64'>'
	with 342778 stored elements in Compressed Sparse Row format>

In [14]:
n_components = 50  # This no. must be the same or smaller than the no. of rows!! Typically set between 100 to 300
svd = TruncatedSVD(n_components=n_components)
tfidf_reduced = svd.fit_transform(tdm)
tfidf_df = pd.DataFrame(tfidf_reduced, columns=[f'component_{i}' for i in range(n_components)])

In [15]:
googleMain.columns

Index(['title', 'description', 'summary', 'installs', 'minInstalls',
       'realInstalls', 'score', 'ratings', 'reviews', 'price', 'free',
       'currency', 'offersIAP', 'inAppProductPrice', 'developer', 'genre',
       'genreId', 'contentRating', 'contentRatingDescription', 'adSupported',
       'released', 'lastUpdatedOn', 'version', 'appId', 'categories_list',
       'min_inAppProductPrice', 'max_inAppProductPrice', '1starrating_no',
       '2starrating_no', '3starrating_no', '4starrating_no', '5starrating_no'],
      dtype='object')

In [16]:
googleMainFiltered= googleMain[['realInstalls', 'score', 'reviews', 'price', 'free', 'genre', 'contentRating', 'adSupported', '1starrating_no', '2starrating_no', '3starrating_no', '4starrating_no', '5starrating_no']].loc[googleMain.title != 'None']
googleMainFiltered.head()

,realInstalls,score,reviews,price,free,genre,contentRating,adSupported,1starrating_no,2starrating_no,3starrating_no,4starrating_no,5starrating_no
0,628,2.6,0,0.0,True,Tools,Everyone,False,3,0,0,0,2
1,1011,3.75,6,2.49,False,Tools,Everyone,False,6,0,0,6,12
2,4556,3.6,3,0.0,True,Tools,Everyone,True,4,0,4,4,9
3,35012,4.2083335,20,0.0,True,Tools,Everyone,True,19,9,7,25,124
4,494071,3.41,103,0.0,True,Tools,Everyone,True,641,131,247,247,1064


In [17]:
# create dummy variables (one hot)
googleMainFiltered['free'] = googleMainFiltered['free'].map({'False':0, 'True':1}).astype('int64')
googleMainFiltered['adSupported'] = googleMainFiltered['adSupported'].map({'False':0, 'True':1}).astype('int64')
googleMainFiltered = pd.concat([googleMainFiltered, pd.get_dummies(googleMainFiltered['genre'], prefix='genre')], axis=1)
googleMainFiltered = pd.concat([googleMainFiltered, pd.get_dummies(googleMainFiltered['contentRating'], prefix='contentRating')], axis=1)
googleMainFiltered.pop('genre')
googleMainFiltered.pop('contentRating')

0           Everyone
1           Everyone
2           Everyone
3           Everyone
4           Everyone
5           Everyone
6               Teen
7           Everyone
8         Mature 17+
9           Everyone
10          Everyone
11          Everyone
12          Everyone
13          Everyone
14          Everyone
15          Everyone
16          Everyone
17          Everyone
18          Everyone
19          Everyone
20          Everyone
21          Everyone
22          Everyone
23          Everyone
24          Everyone
25          Everyone
26          Everyone
27          Everyone
28          Everyone
29          Everyone
30          Everyone
31          Everyone
32          Everyone
33          Everyone
34          Everyone
35          Everyone
36          Everyone
37          Everyone
38              Teen
39          Everyone
40          Everyone
41          Everyone
42          Everyone
43          Everyone
44          Everyone
45              Teen
46          Everyone
47          E

In [18]:
# googleMainFiltered = googleMainFiltered.copy() # option 1 ('realInstalls', 'score', 'reviews', 'price', 'free', 'genre', 'contentRating', 'containsAds')
# googleMainFiltered = tfidf_df # option 2 ('title', 'description', 'summary')
googleMainFiltered = pd.concat([googleMainFiltered, tfidf_df], axis = 1) # option 3 ('realInstalls', 'score', 'reviews', 'price', 'free', 'genre', 'contentRating', 'containsAds' + 'title', 'description', 'summary')

googleMainFiltered.replace('None', np.nan, inplace=True)
googleMainFiltered.replace('nan', np.nan, inplace=True)
googleMainFiltered = googleMainFiltered.dropna()
googleMainFiltered = googleMainFiltered.astype('float64')

In [19]:
googleMainFiltered.shape

(5000, 113)

In [20]:
# import pandas as pd
# from statsmodels.stats.outliers_influence import variance_inflation_factor

# while True:
#     vif = pd.DataFrame()
#     vif["Features"] = googleMainFiltered.columns
#     vif["VIF"] = [variance_inflation_factor(googleMainFiltered.values, i) for i in range(googleMainFiltered.shape[1])]

#     max_vif = vif["VIF"].max()

#     if max_vif > 10:
#         column_to_remove = vif.loc[vif["VIF"].idxmax(), "Features"]
#         googleMainFiltered = googleMainFiltered.drop(column_to_remove, axis=1)
#     else:
#         break

#### K-Nearest Neighbours

In [21]:
# to display the results
def knn_showResults(qind, dis, ind, df_condensed, df):
  for i, d in zip(ind[qind], dis[qind]) :
    #retrieve the original row index, to access 'data'
    reali = df_condensed.loc[i, 'index']
    print('=====')
    print('\tRow Index: ' + str(reali) + '\tDistance: ' + str(d) + '\tApp Title: ' + df.loc[reali, 'title'] )
    #access the values in 'df_condensed' using new row index
    # print(str(df_condensed.loc[i]))
    print(df.loc[reali, 'title'] + "\t"+ df.loc[reali, 'summary'])

#### Cosine-Similarity

In [22]:
# find top k movies most similar to q
def cosine_showResults(q, k, df, tdm):
    print(df['title'][q])
    vals = cosine_similarity(tdm[q], tdm)
    idx_asc = vals.argsort()[0][-k:] #get index of highest sim values
    idx_dsc = idx_asc[::-1] #reverse the indices
    print(idx_dsc)
    flatv = np.sort(vals[0])
    vk_asc = flatv[-k:]
    vk_dsc = vk_asc[::-1] #reverse the list
    if(vk_dsc[1]==0):
        print("No similar movie is found")
    else:
        for v, i in zip(vk_dsc, idx_dsc):
          # print(df['title'][i]+'\t'+str(v)+"\t"+str(i)+"\t"+str(df['index'][i])+'\t'+str(df['genre'][i]))
          print('=====')
          print(df['title'][i]+'\t'+str(v))
          print('\t'+df['summary'][i])

#### Embedding method (Doc2Vec model)

In [89]:
# Create "text" column by concatenating title, description, and summary
googleMain_sparkDF = googleMain_sparkDF.withColumn('text', concat(col('title'), lit(' '), col('description'), lit(' '), col('summary')))

# Tokenize the text and store it in the "text_tokens" column
googleMain_sparkDF = googleMain_sparkDF.withColumn("text_tokens", split(lower("text"), "\s+"))

# Select only the "text_tokens" column and collect it into a list
text_tokens = googleMain_sparkDF.select("text_tokens").rdd.flatMap(lambda x: x).collect()

In [90]:
# Load data into model
tagged_data = [TaggedDocument(d, [i]) for i, d in enumerate(text_tokens)]
model = Doc2Vec(vector_size=64, min_count=2, epochs=40)
model.build_vocab(tagged_data)

# Train the model using our data
model.train(tagged_data, total_examples=model.corpus_count, epochs=40)

## Run model results

#### K-Nearest Neighbours

In [ ]:
scaler = pp.MinMaxScaler()
googleMainFilteredScaled = scaler.fit_transform(googleMainFiltered)

In [ ]:
googleMainFiltered.reset_index(inplace=True)
googleMain.reset_index(inplace=True)

In [ ]:
nbrs_scaled = NearestNeighbors(n_neighbors=6, algorithm='ball_tree').fit(googleMainFilteredScaled)
dis_scaled, ind_scaled = nbrs_scaled.kneighbors(googleMainFilteredScaled)

In [ ]:
knn_showResults(1010, dis_scaled, ind_scaled, googleMainFiltered, googleMain) # 50 # 52

#### Cosine-Similarity

In [ ]:
cosine_showResults(1010, 6, googleMain, tdm) # tdm only

#### Embedding method (Doc2Vec model)

In [91]:
# (DO NOT use this cell! Use the other one below)

newData = """
This application helps you to securely store your data in a vault with a pin code. it can include photos, videos, messages, and also your own notes such as passwords.
"""

test_doc = word_tokenize(newData.lower())
test_vec = model.infer_vector(test_doc)
results = model.docvecs.most_similar(positive=[test_vec], topn=5)

#check the results. Do they make sense?
for i, (doc_id, similarity_score) in enumerate(results):
    title = googleMain_sparkDF.select("title").collect()[doc_id][0]
    text = googleMain_sparkDF.select("text").collect()[doc_id][0]
    print(f"Result {i+1}:\n")
    print(f"Score:\n{similarity_score}\n")
    print(f"Title:\n{title}\n")
    print(f"Details:\n{text}\n")
    print("-" * 50)

Result 1:

Score:
0.603061854839325

Title:
Recover Deleted All Files, Pho

Details:
Recover Deleted All Files, Pho Recover Deleted All Files, Photos and Contacts

Photo & Video Restore or Photo Video Recovery app with Contacts Recovery or Restore Contacts feature is an awesome Recovery app for deleted Photo and Video. This application will help you to restore all the deleted photos, videos , files and contacts with this simple android free app!

Data Recovery Deleted Photos, Video, Files Restore is the newly developed application that helps the user to recover lost data if they got deleted intentional or unintentionally. 

All user has to scan whether the whole device or just a selected and wait for few seconds and the system will automatically search the location and retrieve all items of lost data and it’s up to user now whether to store the data retrieved or permanently delete the data so no other can find the data on your device.

Contact Recovery can restore your deleted contacts

## Chosen method:
#### Embedding method (Doc2Vec model)

In [128]:
# Define the scope of the Google Sheets API
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

# Set the path to the service account key JSON file
service_account_key_path = '/content/drive/MyDrive/big-data-analytics-415801-f8364cc7c7d5.json'

# Authenticate with Google Sheets API using the service account key
credentials = ServiceAccountCredentials.from_json_keyfile_name(service_account_key_path, scope)
gspread_client = gspread.authorize(credentials)

# Open the Google Sheets by its URL
sheet_url = 'https://docs.google.com/spreadsheets/d/1zo96WvtgcfznAmSjlQJpnbKIX_NfSIMpsdLcrJOYctw/edit#gid=0'
spreadsheet = gspread_client.open_by_url(sheet_url)

# Select the specific worksheet by its title or index
worksheet = spreadsheet.sheet1  # Change to the desired sheet

# Read data from the worksheet into a Pandas DataFrame
data = worksheet.get_all_records()
newApplications_df = pd.DataFrame(data)

newApplications_df

,Timestamp,What is the name of your new application?,Please provide a description for your application.,Please provide a short summary for your application.
0,4/8/2024 15:54:24,info vault,This application helps you to securely store y...,"It can include photos, videos, messages, and a..."
1,4/8/2024 15:54:24,pokemon game,this is the best game ever! Features all kinds...,"features all 8 gyms in pokemon across kanto, j..."
2,4/8/2024 15:54:24,test,test2,test3
3,4/8/2024 15:58:44,aaa,bbb,ccc
4,4/8/2024 16:00:26,q,q,q
5,4/8/2024 16:03:00,e,e,e


In [125]:
# Prep data to import into GBQ

df = pd.DataFrame(columns = ["newApp", "appRank", "appName", "appId", "appScore"])

for index, series in newApplications_df.iterrows():

  newData = series['What is the name of your new application?'] + ' ' + \
  series['Please provide a description for your application.'] + ' ' + \
  series['Please provide a short summary for your application.']

  test_doc = word_tokenize(newData.lower())
  test_vec = model.infer_vector(test_doc)
  results = model.docvecs.most_similar(positive=[test_vec], topn=5)

  #check the results. Do they make sense?
  for i, (doc_id, similarity_score) in enumerate(results):
      title = googleMain_sparkDF.select("title").collect()[doc_id][0]
      id = googleMain_sparkDF.select("appId").collect()[doc_id][0]
      text = googleMain_sparkDF.select("text").collect()[doc_id][0]

      row = [series['What is the name of your new application?'], i+1, title, id, similarity_score]
      df.loc[len(df)] = row

In [126]:
df

,newApp,appRank,appName,appId,appScore
0,info vault,1,"Recover Deleted All Files, Pho",recoverdeletedata.deletedmediarecovery,0.602394
1,info vault,2,CCSWE App Manager (Root),com.ccswe.appmanager.root_20201007,0.524441
2,info vault,3,Apk Extractor Fastest & Suppor,com.iraavanan.apkextractor,0.518113
3,info vault,4,Interview Tricks,com.d_raman.interviewtricks,0.516376
4,info vault,5,Uninstaller,ndt.mobile.android.easiestuninstall,0.515673
5,pokemon game,1,Cocos2D-Mono Tests,cocos2d_mono.Tests.Android,0.581391
6,pokemon game,2,大和証券 マイナンバー登録用アプリ,jp.co.dir.ss.moe.client,0.541317
7,pokemon game,3,PredatorSense,com.acer.android.predator.mobile,0.527073
8,pokemon game,4,StickMan Running,com.gameCenter.SpideManRun,0.524887
9,pokemon game,5,Retrobird,tr.com.retrobird.mobil,0.523805


In [ ]:
# Stop Spark session
# spark.stop()